Look up *all* of our experiments to see what documents are attached and what their starting nucleic acid amounts are.

This notebook is painfully slow to run.

The output is an excel sheet, because there's enough data this is a mess.

In [1]:
from __future__ import print_function
from IPython.display import clear_output, display
import pandas
import RDF
from curation_common import *
from htsworkflow.util.rdfhelp import fromTypedNode

In [2]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [3]:
search = server.get_json('/search/?type=experiment&lab.title=Barbara Wold, Caltech')

In [4]:
experiments = [x['@id'] for x in search['@graph']]

In [5]:
len(experiments)

257

In [6]:
model = get_model(use_contexts=False)

In [33]:
s = RDF.Serializer('turtle')

In [37]:
s.serialize_model_to_file('/tmp/rnaseq.turtle', model)

0

In [38]:
model2 = get_model(use_contexts=False)

In [39]:
p = RDF.Parser('turtle')

In [42]:
%timeit p.parse_into_model(model2, 'file:///tmp/rnaseq.turtle')

Caught error "URI 'https://www.encodeproject.org/profiles/MadQualityMetric.json#MAD of log ratios' contains bad character(s)"


RedlandError: "URI 'https://www.encodeproject.org/profiles/MadQualityMetric.json#MAD of log ratios' contains bad character(s)"

In [7]:
for i, e in enumerate(experiments):
    display("loading: {} ({} of {})".format(e, i, len(experiments)))
    jsonld = server.get_jsonld(e)
    load_jsonld_into_model(model, jsonld)
    clear_output()

In [14]:
q = RDF.SPARQLQuery("""
PREFIX htsw: <http://jumpgate.caltech.edu/wiki/LibraryOntology#>
PREFIX barbara-wold: <http://jumpgate.caltech.edu/wiki/LibraryOntology#>
PREFIX experiment: <https://www.encodeproject.org/profiles/Experiment.json#> 
PREFIX library: <https://www.encodeproject.org/profiles/Library.json#> 
PREFIX replicate: <https://www.encodeproject.org/profiles/Replicate.json#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

construct { 
  ?library htsw:library ?jumpgate . 
} where {
  select ?library 
         ( REPLACE(?alias, "barbara-wold:", "http://jumpgate.caltech.edu/library/") as ?short )
         ( IRI(CONCAT(?short, "/"))  as ?jumpgate )
  where {
    ?library library:aliases ?alias .
  }
}""")

count = 0
results = q.execute(model)
for s in results.as_stream():
    jumpgate = s.object
    model.append(s)
    display('loading {} {}'.format(jumpgate, count))
    load_into_model(model, 'rdfa', jumpgate)
    count += 1
    clear_output()
display(count)
 

442

# Find names

In [16]:
%%sparql -m model -c -o library_name
select ?accession ?library ?name
where {
   ?accession a library:Library ;
              htsw:library ?library .
   ?library htsw:name ?name .
}

Found 440 rows.


In [10]:
report = []
report_index = []
for row in library_name:
    record = { k: str(row[k]) for k in row }
    report_index.append(record['accession'])
    del record['accession']
    report.append(record)
library_name_sheet = pandas.DataFrame(report, columns=['library', 'name'], index=report_index)

# Find starting amount

In [20]:
%%sparql -m model -c -o starting_amount
select ?accession ?na_starting ?na_units 
where {
   ?accession a library:Library ;
              library:nucleic_acid_starting_quantity ?na_starting ;
              library:nucleic_acid_starting_quantity_units ?na_units ;
}

Found 341 rows.


In [21]:
report = []
report_index = []
for row in starting_amount:
    na_amount = float(str(row['na_starting'])) if row['na_starting'] else ''
    record = { 
       'na_amount': na_amount,
       'units': str(row['na_units']) if row['na_units'] else '',
    }
    report_index.append(str(row['accession']))

    report.append(record)
start_amount_sheet = pandas.DataFrame(report, columns=['na_amount', 'units'], index=report_index)

# Update nucleic acid starting amount

In [28]:
import gcat

In [ ]:
update_amounts = gcat.get_file("library to nucleic acid starting amount")

In [ ]:
def update_starting_quantity():
    for row in update_amounts:
        if not pandas.isnull(row['na_amount']):
            url = '/library/{}'.format(row['accession'])
            display('checking: {}'.format(url))
            library = server.get_json(url)
            if 'nucleic_acid_starting_quantity' not in library and \
               library['submitted_by'] == '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/' and\
               library['status'] != 'released':
                patch = {'nucleic_acid_starting_quantity': str(row['na_amount']),
                         'nucleic_acid_starting_quantity_units': row['units']}
                print(server.patch_json(url, patch))
            clear_output()
#update_starting_quantity()

# Verify documents

In [ ]:
%%sparql -m model -c -o documents
select ?accession ?doc_alias ?doc_type ?description ?document
where {
   ?accession a library:library ;
              library:documents ?document .

   ?document a <https://www.encodeproject.org/profiles/document.json#document> ;
             <https://www.encodeproject.org/profiles/document.json#document_type> ?doc_type ;
             rdf:description ?description.
   OPTIONAL { ?document <https://www.encodeproject.org/profiles/document.json#aliases> ?doc_alias . }
}


In [ ]:
results = []
results_index = []
for row in documents:
    record = { k: str(row[k]) for k in row}
    results_index.append(record['accession'])
    del record['accession']
    results.append(record)
document_sheet = pandas.DataFrame(results,
                                  columns=['doc_alias', 'doc_type', 'description', 'document'], 
                                  index=results_index)

In [ ]:
%%sparql -m model -o grants
select ?accession ?award ?award_name
where {
   ?accession a library:library ;
              library:award ?award .
   ?award <https://www.encodeproject.org/profiles/award.json#rfa> ?award_name
}
order by ?award_name

In [ ]:
results = []
results_index = []
for row in grants:
    results.append([str(row['award_name'])])
    results_index.append(str(row['accession']))
grants_sheet = pandas.DataFrame(results,
                                columns=['award_name'],
                                index=results_index)

* =HYPERLINK(A2, MID(A2, 41, 11))
* =HYPERLINK(B2, MID(B2, 37, 5))

In [ ]:
document_pivot = document_sheet.reset_index().pivot(index='index', columns='document', values='doc_alias').fillna(value='')

In [ ]:
%%sparql -m model -o spikes
select distinct ?spike
where {
   ?s library:spikeins_used ?spike .
   
}

In [ ]:
for row in spikes:
    spike = str(row['spike'])
    jsonld = server.get_jsonld(spike)
    load_jsonld_into_model(model, jsonld)

In [ ]:
%%sparql -m model -c -o spikes
select distinct ?accession ?spike
where {
   ?accession a library:library ;
              library:spikeins_used ?spike .   
}


In [ ]:
results = {}
for row in spikes :
    accession = str(row['accession'])
    s = str(row['spike'])
    record = results.get(accession, {})
    if s == 'https://www.encodeproject.org/datasets/ENCSR013YHQ/':
        record['profile 4'] = 'barbara-wold:SpikeProfile4'
    elif s == 'https://www.encodeproject.org/datasets/ENCSR133ALU/':
        record['single'] = 'ERCC-single'
    elif s == 'https://www.encodeproject.org/datasets/ENCSR156CIL/':
        record['gingeras'] = 'thomas-gingeras:AmbionMix1'
    elif s == 'https://www.encodeproject.org/datasets/ENCSR193ZXE/':
        record['profile 3'] = 'barbara-wold:SpikesProfile3'
    results[accession] = record
spikes_sheet = pandas.DataFrame([ results[k] for k in results ],
                                columns=['profile 3', 'profile 4', 'gingeras', 'single'],
                                index=results.keys()
                                )

In [ ]:
#document_pivot = document_sheet.pivot(index='accession', columns='document', values='doc_alias').fillna(value='')
spikes_sheet.fillna(value='')

In [ ]:
name_award = pandas.merge(library_name_sheet, grants_sheet, left_index=True, right_index=True, how='outer')
name_award_amounts = pandas.merge(name_award, start_amount_sheet, left_index=True, right_index=True, how='outer')
name_award_documents = pandas.merge(name_award, document_pivot, left_index=True, right_index=True, how='outer')
name_award_spikes = pandas.merge(name_award, spikes_sheet, left_index=True, right_index=True, how='outer')

name_award_xlsx = pandas.ExcelWriter('/tmp/name_award.xlsx')
name_award_amounts.to_excel(name_award_xlsx, 'starting amounts')
name_award_documents.to_excel(name_award_xlsx, 'documents')
name_award_spikes.to_excel(name_award_xlsx, 'spikes')
name_award_xlsx.save()

# Make 13 pg dataset

In [ ]:
pg_libraries = start_amount_sheet[(start_amount_sheet['na_amount'] == 13) & (start_amount_sheet['units'] == 'pg')]

In [ ]:
%lsns

In [ ]:
pg_files = []
for library in pg_libraries.index:
    q = """
PREFIX replicate: <https://www.encodeproject.org/profiles/replicate.json#>
PREFIX experiment: <https://www.encodeproject.org/profiles/experiment.json#>
select ?file
where {{
  ?rep replicate:library <{library}> ;
       replicate:experiment ?experiment .
  ?experiment experiment:files ?file .
}}
""".format(library=library)
    query = RDF.SPARQLQuery(q)
    for row in query.execute(model):
        pg_files.append(str(row['file'])[len('https://www.encodeproject.org'):])

In [ ]:
pg_files[0]

In [ ]:
pg_dataset = {
  'award': 'U54HG006998',
  'aliases': ['barbara-wold:13-pg-experiments'],
  'lab': 'barbara-wold',
  'description': '13 picogram RNA-Seq experiments',
  'dataset_type': 'project',
  'related_files': pg_files
}

In [ ]:
server.validate(pg_dataset, 'dataset')

In [ ]:
pg_accession = 'ENCSR901QHQ'
if not pg_accession:
    print(server.post_json('/dataset/', pg_dataset))

# Fill in ENCSR112CAL

In [ ]:
name_award_amounts[name_award_amounts['na_amount'] == 1]